In [1]:
#Definindo finais de semana
mes = "07"
lista_fds = ['06','07','13','14','20','21','27','28']

# Biblioteca e Funções

In [2]:
####################################-BIBLIOTECAS-####################################

#manipulação de dados
import pandas as pd
import numpy as np

#string e data
import unidecode #limpeza
import re #regex

#tempo
import time
from time import sleep

#web scraping
import requests
from bs4 import BeautifulSoup

#automação
from selenium import webdriver #navegador
from selenium.webdriver.common.keys import Keys #comandos do teclado
from selenium.webdriver.common.by import By 
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait #esperar
from selenium.webdriver.support.expected_conditions import (visibility_of, 
                                                           staleness_of, 
                                                           invisibility_of_element)
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pyautogui #

import warnings #Warning inuteis
from pandas.errors import SettingWithCopyWarning
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

#manipulação do diretorio
import os

In [3]:
# FUNÇÕES XPATH
def xpath_element(xpath):
    return browser.find_element("xpath", xpath)

def xpath_click(xpath):
    try:
        xpath_element(xpath).click()
    except:
        sleep(2)
        xpath_element(xpath).click()
    
def xpath_send_keys(xpath, texto):
    xpath_element(xpath).send_keys(texto)
    
def xpath_clear(xpath):
    xpath_element(xpath).clear()

def existencia(caminho):
    elementos = browser.find_elements(By.XPATH, caminho)
    return bool(elementos)

In [4]:
#FUNÇÕES
def count_freq(data_frame, coluna):
    freq = df[coluna].value_counts()
    perc = df[coluna].value_counts(normalize = True)*100
    tabela = pd.DataFrame({"Freq":freq, "Perc":perc})
    return display(tabela)

def salvando_nome_arquivo(nome_arquivo, dataframe):
    if os.path.exists(nome_arquivo): #O nome_arquivo existe
        df1 = pd.read_excel(nome_arquivo)
        len_antigo = len(dataframe)
        
        dataframe = pd.concat([df1, dataframe]).reset_index(drop=True)
        dataframe = dataframe.drop_duplicates(subset=['chat'])
        
        print(len(df1), " += ", len_antigo, "=>", len(dataframe))
        
        dataframe.to_excel(nome_arquivo, index=False)
    
    else: #O nome_arquivo não existe
        dataframe.to_excel(nome_arquivo, index=False)
        print(len(dataframe)) 

# ===============================================

# Ligando navegador

In [5]:
#browser.quit(); sleep(5)
browser = webdriver.Firefox()

#fazendo login
browser.get('https://www.pegaplantao.com.br/login')
sleep(1)
xpath_send_keys('//*[@id="MainContent_LoginUser_UserName"]', 'gabriel_s_anjos@yahoo.com') #e-mail
xpath_send_keys('//*[@id="MainContent_LoginUser_Password"]', 'Gabriel2022!') #senha
xpath_click('//*[@id="MainContent_LoginUser_btnLogin"]') #login

In [6]:
browser.get('https://www.pegaplantao.com.br/EscalaMensal/old')

In [8]:
#click setor/profissional
xpath_click('//*[@id="s2id_autogen1"]/a/span[1]')

#click [profissional
xpath_click('//*[@id="select2-drop"]/ul/li[2]/div')

# Capturando setores:

In [9]:
#click criar novo plantão
def criar_plantao():
    xpath_click('//*[@id="shift-monthly"]/div[2]/table/tbody/tr[1]/td[4]/button')
criar_plantao()

def esperar_elemento_aparecer(elemento, tempo):
    sleep(0.5)
    WebDriverWait(browser, tempo).until(visibility_of(xpath_element(elemento)))

sleep(2)
esperar_elemento_aparecer('//*[@id="service-ckb-list-all-owner"]', 10)
print('apareceu')

def esperar_elemento_desaparecer(elemento, tempo):
    sleep(0.5)
    WebDriverWait(browser, tempo).until_not(visibility_of(xpath_element(elemento)))

apareceu


In [10]:
#CAPTURANDO DADOS DO UNIDADE E SETOR
xpath_click('//*[@id="s2id_ddlServiceSector"]/a/span[1]')
setor_html = xpath_element('//*[@id="select2-drop"]/ul').get_attribute('outerHTML')
soup = BeautifulSoup(setor_html, 'html.parser')

lista_unidade = []
lista_unidade_count = []
lista_plt = []
lista_plt_count = []

count_unidade = 1

for i in soup.find_all(class_="select2-results-dept-0 select2-result select2-result-unselectable select2-result-with-children"):
    #print('\n', i.find(class_='select2-result-label').get_text(), '====== \n')
    count_plt = 1
    for t in i.find_all(class_='select2-results-dept-1 select2-result select2-result-selectable'): 
        #print(t.get_text())
        lista_plt.append(t.get_text())
        lista_plt_count.append(count_plt)
        count_plt += 1

        lista_unidade.append(i.find(class_='select2-result-label').get_text())
        lista_unidade_count.append(count_unidade)
    count_unidade += 1

scrap_unidades = pd.DataFrame({'unidade':lista_unidade, 'momento':lista_plt,
                               'unidade_count':lista_unidade_count, 'plt_count':lista_plt_count})

scrap_unidades['momento'] = scrap_unidades['momento'].apply(lambda x: x.replace('  ',' '))

scrap_unidades['momento'] = scrap_unidades['momento'].apply(lambda x: unidecode.unidecode(x.replace('  ',' ').upper().strip()))
scrap_unidades['unidade'] = scrap_unidades['unidade'].apply(lambda x: unidecode.unidecode(x.replace('  ',' ').upper().strip()))
print('OK')

OK


In [11]:
#CARREGANDO OS DADOS PARA SUBIR NO PEGA PLANTÃO
df = pd.read_excel('df_gabs.xlsx') # df_gabs # df_continuar
df = df.sort_values(['nome', 'data']).reset_index(drop=True)

df['setor'][(df['setor'] == 'FDS - C.CIRURGICO ROTINA') & (df['unidade'] == 'SANTA ISABEL')] = 'FDS - MANHA - C.CIRURGICO'

print(len(df))
df.sample(3)

493


,chat,dia,nome,mensagem,controle,data,unidade,extra,tipo,ENTRADA,SAIDA,DURAÇÃO,PERIODO,periodo,setor,sem_ou_fds,unidade2
241,[24/07/2024 09:02:32] Luiz Augusto Xavier Bran...,24/07/2024,LUIZ AUGUSTO XAVIER BRANDAO,24/07 OSASCO LUIZ AUGUSTO ENTRADA 07:00,NaN,24/07/2024,OSASCO,NaN,ENTRADA,07:00,13:00,06:00,manha,2,SEMANA - MANHA - C.CIRURGICO,semana,OSASCO
69,[24/07/2024 14:23:14] Darlan Junior Schneider:...,24/07/2024,DARLAN JUNIOR SCHNEIDER,24/07 OSASCO - DARLAN ENTRADA 09:00,NaN,24/07/2024,OSASCO,NaN,ENTRADA,09:00,13:00,04:00,manha,2,SEMANA - MANHA - C.CIRURGICO,semana,OSASCO
414,[25/07/2024 11:15:51] Rodrigo Miyashiro Nunes ...,25/07/2024,RODRIGO MIYASHIRO NUNES DOS SANTOS,25/07 MORUMBI - MIYASHIRO ENTRADA 11:00,NaN,25/07/2024,MORUMBI,NaN,ENTRADA,11:00,13:00,02:00,manha,2,SEMANA - MANHA - C.CIRURGICO,semana,MORUMBI


## Pareando Setores

In [12]:
setores = pd.read_excel("LocaiseSetores.xlsx")
print(setores.shape)

setores = setores.dropna() #
setores = setores.reset_index(drop=True)

for coluna in ['Local', 'Setor', 'Mudou para:']:
    setores[coluna] = setores[coluna].apply(lambda x: unidecode.unidecode(x).upper().strip())
    setores[coluna] = setores[coluna].apply(lambda x: x.replace('  ', ' '))
    setores[coluna] = setores[coluna].apply(lambda x: x.replace('  ', ' '))

print(setores.shape)
setores.sample(5)

(230, 5)
(217, 5)


,Código,Local,Setor,Situação,Mudou para:
60,9.0,OSASCO,SEMANA - MANHA - COBERTURA EXTRA,Habilitado,04.3 - SEMANA - COBERTURA EXTRA
156,1.0,ANALIA,FDS - D3 - SEGUNDO (B),Habilitado,15 - FDS - D3 (B)
183,10.0,ALPHA,FDS - D1,Habilitado,19 - FDS - D1
202,0.0,ADMINISTRACAO,PERIODO DE DESCANSO - 4,Habilitado,PERIODO DE DESCANSO - 4
66,6.0,MATERNIDADE,SEMANA - TARDE - COBERTURA EXTRA,Habilitado,04.4 - SEMANA - COBERTURA EXTRA


In [13]:
df['setor_antigo'] = df['setor']

for unidade_df in df['unidade'].unique():
    for setor_df in df['setor'][df['unidade'] == unidade_df].unique():
    
        setores_filter = setores[(setores['Local'] == unidade_df) & ((setores['Setor'] == setor_df) | (setores['Mudou para:'] == setor_df))]
        if len(setores_filter) >= 1:
            #print(unidade_df, '|', setor_df, '===========================')
            #print('encontrado')

            df['setor'][(df['unidade'] == unidade_df) & (df['setor'] == setor_df)] = setores['Mudou para:'][setores_filter.index.values[0]]

            pass
        else:
            print(unidade_df, '|', setor_df, '=!=!=!=!=!=!=!=!=!=!=!=!=!=!')
            print('não encontrado')

In [14]:
#TESTANDO COMPATIBILIDADE DOS DADOS CAPTURADOS E DADOS CARREGADOS
for X in range(0, len(df)):
    if len(scrap_unidades.loc[(scrap_unidades.unidade == df['unidade'][X]) & 
                                 (scrap_unidades.momento == df['setor'][X])]) < 1:
        print(X)
        print(df['unidade'][X], '-', df['setor'][X])
        #print('index', ind)
        #print(scrap_unidades['unidade'][ind], '-', scrap_unidades['momento'][ind])
        #print('codigos:', unidade, setorzin)
        

print("="*20, "TUDO OK", "="*20)

==================== TUDO OK ====================


In [15]:
print(df['setor'].unique())

['04 - SEMANA - CENTRO CIRURGICO' '07 - NOTURNA - SAIDA APOS 19HS'
 '01 - NOTURNA - ENTRADA ANTES DAS 07HS'
 '04.1 - SEMANA - CENTRO CIRURGICO']


In [16]:
print(df['setor'].unique())

df['setor_inicio'] = np.nan
for i in range(len(df)):
    corte = df['setor'][i].find('-')
    texto = df['setor'][i][:corte].strip()
    df['setor_inicio'][i] = texto

print(df['setor_inicio'].unique())

['04 - SEMANA - CENTRO CIRURGICO' '07 - NOTURNA - SAIDA APOS 19HS'
 '01 - NOTURNA - ENTRADA ANTES DAS 07HS'
 '04.1 - SEMANA - CENTRO CIRURGICO']
['04' '07' '01' '04.1']


# Funções Selenium

In [17]:
def subir_pg():
    browser.execute_script("window.scrollTo(0, 0);")

In [18]:
def reset_selecao():
    #xpath_click('/html')#; sleep(0.3)
    #xpath_click('/html')

    posicao_antiga = pyautogui.position() 
    pyautogui.click(x=200, y=158); sleep(0.3)
    pyautogui.click(x=200, y=158)
    pyautogui.moveTo(posicao_antiga) 


def fechar_caixa_de_selecao():
    # Localiza um elemento que é seguro para clicar (por exemplo, o corpo da página)
    elemento_seguro = browser.find_element(By.TAG_NAME, 'body')
    # Cria uma ação para clicar no elemento seguro
    ActionChains(browser).move_to_element(elemento_seguro).click().perform()


def reset_selecao1():
    reset_selecao2()
    #fechar_caixa_de_selecao(); sleep(0.3)
    #fechar_caixa_de_selecao()

    posicao_antiga = pyautogui.position() 
    pyautogui.click(x=200, y=158); sleep(0.3)
    pyautogui.click(x=200, y=158)
    pyautogui.moveTo(posicao_antiga)


def reset_selecao2():
    posicao_antiga = pyautogui.position() 
    pyautogui.click(x=200, y=158); sleep(0.3)
    pyautogui.click(x=200, y=158)
    pyautogui.moveTo(posicao_antiga) 
reset_selecao2()

In [19]:
#selecionando setor
def setor(unidade, setor):
    #print('base:', unidade, '-', setor)
    xpath_click('//*[@id="s2id_ddlServiceSector"]/a/span[1]')
    ind = scrap_unidades.loc[(scrap_unidades.unidade == unidade) & 
                             (scrap_unidades.momento == setor)].index.values[0]
    unidade_sel = scrap_unidades['unidade_count'][ind]
    setor_sel = scrap_unidades['plt_count'][ind]
    xpath_click('//*[@id="select2-drop"]/ul/li['+str(unidade_sel)+']/ul/li['+str(setor_sel)+']/div')   
setor("SANTA ISABEL", '04 - SEMANA - CENTRO CIRURGICO')

#inserir e selecionar profissional
def nome_func(nome_profissional):
    #listar todos
    xpath_click('//*[@id="service-ckb-list-all-owner"]')
    
    xpath_click('//*[@id="service-ckb-list-all-owner"]')
    xpath_click('//*[@id="service-ckb-list-all-owner"]')
    sleep(0.3)
    xpath_click('//*[@id="s2id_ddlProfessionalA"]/a/span[1]')
    xpath_send_keys('//*[@id="select2-drop"]/div/input', nome_profissional)
    #Dando enter
    xpath_send_keys('//*[@id="select2-drop"]/div/input', Keys.RETURN)
nome_func('OTTO MITTERMAYER')

#inserir horario e duração
def hora(inicio, duracao):
    #Inicio do plantão
    xpath_clear('//*[@id="txtServiceStartTime"]')
    xpath_send_keys('//*[@id="txtServiceStartTime"]', inicio)
    sleep(0.3)
    xpath_clear('//*[@id="txtServiceDuration"]')
    xpath_send_keys('//*[@id="txtServiceDuration"]', duracao)
hora('13:30', '05:00')

In [20]:
#inserir data
def data_func(data_selecionada):
    data_selecionada = str(data_selecionada)
    i_chato2 = str(5) #4 e 5
    i_chato = str(3) #2 e 3

    xpath_clear('/html/body/div['+i_chato2+']/div/div[2]/div[2]/div[5]/div[1]/div[2]/div/div[2]/div[1]/div['+i_chato+']/input[1]')
    #xpath_clear('/html/body/div[4]/div/div[2]/div[2]/div[5]/div[1]/div[2]/div/div[2]/div[1]/div[3]/input[2]')
    sleep(0.3)
    
    xpath_send_keys('/html/body/div['+i_chato2+']/div/div[2]/div[2]/div[5]/div[1]/div[2]/div/div[2]/div[1]/div['+i_chato+']/input[1]', data_selecionada)
    #xpath_send_keys('/html/body/div[4]/div/div[2]/div[2]/div[5]/div[1]/div[2]/div/div[2]/div[1]/div[3]/input[2]', data_selecionada)
    reset_selecao2()
data_func("15/07/2024")

In [21]:
def tipo_registro_func(i):
    xpath_click('//*[@id="s2id_add-service-service-types"]/a/span[1]')
    sleep(0.5)

    xpaths = [
        '/html/body/div[12]/div/input',
        '/html/body/div[11]/div/input',
        '/html/body/div[10]/div/input',
        '/html/body/div[9]/div/input',
        '/html/body/div[8]/div/input',
        '/html/body/div[7]/div/input',
        '/html/body/div[6]/div/input',
    ]

    for xpath in xpaths:
        try:
            xpath_send_keys(xpath, i)
            sleep(1)
            xpath_para_clicar = xpath[:xpath.find(']')+1] + '/ul/li/div'
            xpath_click(xpath_para_clicar)
            return  # Sai da função se encontrar um xpath que funcione
        except:
            continue  # Tenta o próximo xpath se ocorrer uma exceção
    
    print('erro')  # Imprime 'erro' se nenhum xpath funcionar

tipo_registro_func('VOLUNTÁRIO')

In [39]:
#inserindo pontuação
def pontuacao(pontos): # NÃO SE ALTERA MAIS
    #xpath_clear('//*[@id="txtServiceValue"]')
    #xpath_send_keys('//*[@id="txtServiceValue"]', pontos) #Pontos
    pass

pontuacao("666")

#selecionar tipo de plantão e ajustando tempo
def tipo_do_registro(duracao, sem_ou_fds, tipo_registro):
    if tipo_registro == 'extra':
        print('EXTRA')
        sleep(0.5)
        reset_selecao2()
        sleep(1)
        tipo_registro_func('VOLUNTÁRIO') #escrevendo
        if int(duracao[:2]) >= 5 and sem_ou_fds == "semana":
            pontuacao(200)
            pontuacao(200)
            print('duração >= 5 => 200 pontos')
        else:
            pass
    elif tipo_registro == 'DESLOCADO':
        pass
        #print('DESLOCADO')
        #sleep(1)
        #reset_selecao2()
        
        #xpath_click('//*[@id="s2id_add-service-service-types"]/a/span[1]')
        #xpath_send_keys('/html/body/div[11]/div/input', 'VOLUNTÁRIO') #escrevendo
        #xpath_click('/html/body/div[11]/ul/li/div') #clicando no primeiro
        #if int(duracao[:2]) >= 5 and sem_ou_fds == "semana":
        #    pontuacao(100)
        #    pontuacao(100)
        #    print('duração >5 -> 100 pontos')
        #else:
        #    pass
        #sleep(1)
    else:
        if int(duracao[:2]) >= 5 and sem_ou_fds == "semana":
            pontuacao(100)
            pontuacao(100)
            print('duração >5 -> 100 pontos')
        else:
            pass
        sleep(1)

#tipo_do_registro("05:00", "semana", "extra")
#tipo_do_registro(df['DURAÇÃO'][h], df['sem_ou_fds'][h], df['extra'][h])

In [40]:
def apagando_registro():
    chatao = str(5) #4 ou 5
    #Apagando
    browser.find_element("xpath", '/html/body/div['+chatao+']/div/div[1]/div[2]/button[1]').click()
    #Clicando em continuar
    sleep(0.5)
    browser.find_element("xpath", '/html/body/div['+chatao+']/div/div[1]/div[3]/button[2]').click()
#apagando_registro()

In [24]:
def insistindo_no_xpath(xpathzinho):
    xpath_click(xpathzinho)
    try:
        esperar_elemento_desaparecer(xpathzinho, 10)
    except:
        xpath_click(xpathzinho)

#########################################################

#salvando plantão
def salvar_plantao():
    xpathzinho = '//*[@id="modal-add-service"]/div/div[1]/div[2]/button[4]'
    xpath_click(xpathzinho)
#salvar_plantao()

#fechando plantão
def fechar():
    xpathzinho = '//*[@id="modal-add-service"]/div/div[1]/button'
    xpath_click(xpathzinho)
#fechar()

#atualizando plantão
xpath_atualizar = '/html/body/div[5]/div/div[1]/div[2]/button[5]'
#xpath_atualizar = '/html/body/div[4]/div/div[1]/div[2]/button[5]'
def atualizando_plantao():
    xpath_click(xpath_atualizar)
atualizando_plantao()

# Função Tabela

In [25]:
######### ######### FUNÇÃO DE CAPTURA ######### ######### 

#pacotes para manipular data
from datetime import date
from datetime import time
from datetime import datetime
from datetime import timedelta

#capturando registros
unidades = [
            'ADMINISTRACAO','ALPHA','ANALIA',
            'CENTRAL DO TATUAPE',
            'HOSPITAIS EXTERNOS','HOSPITAL PARA TESTE',
            'ITAIM',
            'JABAQUARA',
            'MATERNIDADE','MORUMBI',
            'OSASCO',
            'SANTA ISABEL',
            'VILLA','VNS'
            ]

def funcao():
    lista_tabela = []
    for i in range(1, 6): #passando por cada semana[1 a 5]
    ##print('semana:', i, "=="*20)
        for t in range(1, 8): #passando por cada dia[1 a 7] dentro da semana
            reg = xpath_element('/html/body/form/main/div[2]/div[2]/table/tbody/tr['+str(i)+']/td['+str(t)+']').text
            content = reg.splitlines() #deixando as palavras em um item dentro de uma lista
            content = [unidecode.unidecode(i.strip().upper()) for i in content] #tratando cada palavra

            dia = content[0] #dia do registro
            del(content[0]) #deletando o dia, ja que foi coletado
            del(content[-1]) #deletando residuo do wepscraping

            if len(content) != 0: #ignorando dias em branco (sem registro) ¹
                #print('dia:', t)

                ##print('dia:', dia)
                #print(content)

                count_hora = 0 #count para capturar a hora de entrada e saida, count=1(entrada) | count=2(saida)
                lista_control = []
                for y in content: #passando um for sobre cada item da lista
                    if y in unidades: # capturando a unidade e dando inicio a novo registro novo
                        #print('================unidade:', y)
                        unidade = y
                    elif y[2] == ':': #reconhecendo se o item é hora (HH*:*mm)
                        count_hora += 1
                        if count_hora == 1: #capturando entrada
                            #print('hora entrada:', y)
                            lista_control.append(y)
                        elif count_hora == 2: #capturando saida
                            #print('hora saida:', y)
                            lista_control.append(y)
                            ##print('lista teste', lista_control)
                            lista_tabela.append(lista_control)
                            lista_control = []
                            count_hora = 0
                    else: #capturando dia, unidade, e periodo
                        #print('.')
                        #print('periodo:', y)
                        lista_control.append(dia)
                        lista_control.append(unidade)
                        lista_control.append(y)
                        lista_control.append(i)
                        lista_control.append(t) #dia semanal

                ##print('='*20)    

            else:
                continue #ignorando dias em branco (sem registro) ²
    
    tabela = pd.DataFrame(lista_tabela, columns = ['Dia','Unidade','Periodo','Semana','Dia Semana','Entrada','Saida'])
    
    #limpeza da tabela
    #########3
    tabela['Dia'] = tabela['Dia'].apply(lambda x: int(x))

    #recolhendo os dias do mes passado que ainda aparecem na primeira semana
    dias_mes_passado = list(tabela[(tabela['Dia'] > 20) & (tabela['Semana'] == 1)].index) 
    tabela.drop(dias_mes_passado, axis=0, inplace=True) #excluindo
    tabela.reset_index(drop=True, inplace=True) #resetando index

    #Inserindo a ordem dos registros
    tabela['ordem'] = ''

    for i in tabela['Dia'].unique():
        for t in range(0, len(tabela[tabela['Dia'] == i].index)):
            tabela['ordem'][tabela[tabela['Dia'] == i].index[t]] = t+1

    #Inserindo coluna de data        
    tabela['data'] = tabela['Dia'].apply(lambda x: str(x))

    for i in range(len(tabela)):
        if len(tabela['data'][i]) == 1:
            tabela['data'][i] = "0" + tabela['data'][i]

        tabela['data'][i] = tabela['data'][i] + "/" + mes + "/2024" 

    tabela = tabela.query("Unidade != 'ADMINISTRACAO'")
    tabela = tabela[~((tabela['Semana'] >= 5) & (tabela['Dia'] < 10))]
    tabela.reset_index(drop=True, inplace=True) #resetando index
    
    tabela['PERIODO'] = np.nan
    for i in range(0, len(tabela)):
        if int(tabela['Entrada'][i][:2]) <= 6:
            tabela['PERIODO'][i] = '6h'
        elif int(tabela['Entrada'][i][:2]) >= 7 and int(tabela['Entrada'][i][:2]) <= 12:
            tabela['PERIODO'][i] = 'manha'
        elif int(tabela['Entrada'][i][:2]) >= 13 and int(tabela['Entrada'][i][:2]) <= 18:
            tabela['PERIODO'][i] = 'tarde'
        else:
            tabela['PERIODO'][i] = 'noturno saida'

    tabela['periodo'] = np.nan
    tabela['periodo'][tabela['PERIODO'] == '6h'] = 1
    tabela['periodo'][tabela['PERIODO'] == 'manha'] = 2
    tabela['periodo'][tabela['PERIODO'] == 'tarde'] = 3
    tabela['periodo'][tabela['PERIODO'] == 'noturno saida'] = 4
    
    return tabela
    
tabela = funcao()
tabela.head()

,Dia,Unidade,Periodo,Semana,Dia Semana,Entrada,Saida,ordem,data,PERIODO,periodo
0,3,ANALIA,01 - NOTURNA - ENTRADA ANTES DAS 07HS,1,3,06:00,07:00,1,03/07/2024,6h,1.0
1,3,ANALIA,04 - SEMANA - CENTRO CIRURGICO,1,3,07:00,13:00,2,03/07/2024,manha,2.0
2,3,ANALIA,04.1 - SEMANA - CENTRO CIRURGICO,1,3,13:00,19:00,3,03/07/2024,tarde,3.0
3,3,ANALIA,07 - NOTURNA - SAIDA APOS 19HS,1,3,19:00,20:00,4,03/07/2024,noturno saida,4.0
4,4,ANALIA,01 - NOTURNA - ENTRADA ANTES DAS 07HS,1,4,06:00,07:00,1,04/07/2024,6h,1.0


# CAPTURANDO DADOS

## Nomes

In [26]:
def diferenca_horas(horario_inicial, horario_final):
    formato = "%H:%M"
    horario_inicial = datetime.strptime(horario_inicial, formato)
    horario_final = datetime.strptime(horario_final, formato)
    diferenca = horario_final - horario_inicial
    if diferenca.days < 0:
        diferenca = horario_inicial - horario_final
        return diferenca.total_seconds() / 60 * (-1)
    else:
        return diferenca.total_seconds() / 60

In [27]:
# Selecionar nome
def xpath_estranho(i):
    xpaths = [
        '/html/body/div[12]/div/input',
        '/html/body/div[11]/div/input',
        '/html/body/div[10]/div/input',
        '/html/body/div[9]/div/input',
        '/html/body/div[8]/div/input',
        '/html/body/div[7]/div/input',
        '/html/body/div[6]/div/input',
    ]
    
    for xpath in xpaths:
        try:
            xpath_send_keys(xpath, i)
            return  # Sai da função se encontrar um xpath que funcione
        except:
            continue  # Tenta o próximo xpath se ocorrer uma exceção
    
    print('erro')  # Imprime 'erro' se nenhum xpath funcionar
#xpath_estranho(i)

In [28]:
xpath_click('//*[@id="s2id_ddlProfessionals"]/a') #barra de nome
sleep(2)

html = browser.find_element("xpath", '//*[@id="select2-drop"]/ul').get_attribute('outerHTML') #
soup_nome = BeautifulSoup(html, 'html.parser')
#soup_nome
NOMES = []
for i in soup_nome.find_all(class_ ="select2-results-dept-0 select2-result select2-result-selectable"):
    #print(i.get_text())
    NOMES.append(i.get_text())
#del(NOMES[0])

reset_selecao()
display(NOMES)
print(len(NOMES))

['Nada Selecionado',
 'ALEXANDRE AMARAL LUCAS',
 'ALEXANDRE BERNARDINO',
 'ALEXSANDER TAKASHI HAMADA',
 'ALINE FRANCHINI GOULART LIMA',
 'AMANDA KIYOKO SOUZA KOSSAKA',
 'ANA CLARA MOURÃO BARRETO',
 'ANA GABRIELA GEWEHR VALE SALMEN',
 'ANA JUNG DOCKO',
 'ANDRÉ AUGUSTO BELLINI',
 'ANDRÉ DA COSTA AGUIAR VENTURA',
 'ANDRÉ LUIS XAVIER FRANCO',
 'ANDRÉ MORTARI PLÁ GIL',
 'ANDRÉIA LOUREIRO MORI',
 'ANNA BEATRIZ ARANHA TREVISAN',
 'ANTONIO ALVARO DIAS LIMA DA CUNHA NETO',
 'ARTHUR OLIVEIRA FACCHINI',
 'ARTHUR RODRIGUES TORRELIO',
 'AUGUSTO DE CASTRO SANTOS QUEIROZ',
 'BEATRIZ ANTIPOU DOS SANTOS',
 'BIANCA PRISCILA BUNIOTTI',
 'BRIAN SOSSAI PACHECO',
 'BRUCE YE MAN CHOW',
 'BRUNA DAIBERT DANESIN',
 'BRUNO FERRAZ DE CAMPOS BARONE',
 'BRUNO STORNIOLI TAGUCHI LEMOS',
 'CAIO TULIO DUARTE',
 'CAMILA CHIARANTANO HARRISON',
 'CAMILA RESENDE DE PAULA',
 'CARLOS AUGUSTO TREVISAN',
 'CARLOS EDUARDO CORRÊA GRILO',
 'CARLOS EDUARDO GONDIM OLIVEIRA',
 'CAROLINE FIRMINO LEMES DA SILVA',
 'CLAUDIO MUELLER KAK

240


In [29]:
len(df['nome'].unique())

178

In [30]:
count_control = 0

tabelona = pd.DataFrame(columns=['Nome'] + list(funcao().columns))
tabelona

,Nome,Dia,Unidade,Periodo,Semana,Dia Semana,Entrada,Saida,ordem,data,PERIODO,periodo


In [31]:
count_nome = 0 

#for NOME in NOMES[count_control:]:
for NOME in df['nome'].unique():

    if count_control > 0:
        if NOMES[count_control] != NOMES[count_control-1] or count_nome == 0:
            print(NOMES[count_control]) 
            #print('\n', '='*20, NOMES[count_control], '='*20, '\n')
            print(count_control, '|', len(NOMES)-count_control)

    xpath_click('//*[@id="s2id_ddlProfessionals"]/a/span[1]'); sleep(1)
    xpath_estranho(NOME)
    xpath_click('//*[@id="select2-drop"]/ul/li/div')

    WebDriverWait(browser, 30).until_not(visibility_of(xpath_element('//*[@id="loader"]')))
    
    tabela = funcao()
    tabela['Nome'] = NOME
    tabelona = pd.concat([tabelona, tabela])
    count_control += 1

ALEXANDRE AMARAL LUCAS
1 | 239
ALEXANDRE BERNARDINO
2 | 238
ALEXSANDER TAKASHI HAMADA
3 | 237
ALINE FRANCHINI GOULART LIMA
4 | 236
AMANDA KIYOKO SOUZA KOSSAKA
5 | 235
ANA CLARA MOURÃO BARRETO
6 | 234
ANA GABRIELA GEWEHR VALE SALMEN
7 | 233
ANA JUNG DOCKO
8 | 232
ANDRÉ AUGUSTO BELLINI
9 | 231
ANDRÉ DA COSTA AGUIAR VENTURA
10 | 230
ANDRÉ LUIS XAVIER FRANCO
11 | 229
ANDRÉ MORTARI PLÁ GIL
12 | 228
ANDRÉIA LOUREIRO MORI
13 | 227
ANNA BEATRIZ ARANHA TREVISAN
14 | 226
ANTONIO ALVARO DIAS LIMA DA CUNHA NETO
15 | 225
ARTHUR OLIVEIRA FACCHINI
16 | 224
ARTHUR RODRIGUES TORRELIO
17 | 223
AUGUSTO DE CASTRO SANTOS QUEIROZ
18 | 222
BEATRIZ ANTIPOU DOS SANTOS
19 | 221
BIANCA PRISCILA BUNIOTTI
20 | 220
BRIAN SOSSAI PACHECO
21 | 219
BRUCE YE MAN CHOW
22 | 218
BRUNA DAIBERT DANESIN
23 | 217
BRUNO FERRAZ DE CAMPOS BARONE
24 | 216
BRUNO STORNIOLI TAGUCHI LEMOS
25 | 215
CAIO TULIO DUARTE
26 | 214
CAMILA CHIARANTANO HARRISON
27 | 213
CAMILA RESENDE DE PAULA
28 | 212
CARLOS AUGUSTO TREVISAN
29 | 211
CARLOS ED

In [32]:
tabelona.reset_index(drop=True, inplace=True)

print(tabelona['Periodo'].unique())

tabelona['setor_inicio'] = np.nan
for i in range(len(tabelona)):
    corte = tabelona['Periodo'][i].find('-')
    texto = tabelona['Periodo'][i][:corte].strip()
    tabelona['setor_inicio'][i] = str(texto)

print(tabelona['setor_inicio'].unique())

['01 - NOTURNA - ENTRADA ANTES DAS 07HS' '04 - SEMANA - CENTRO CIRURGICO'
 '04.1 - SEMANA - CENTRO CIRURGICO' '07 - NOTURNA - SAIDA APOS 19HS'
 '08 - NOTURNA - DISTANCIA' '13 - FDS - CENTRO CIRURGICO'
 '14 - FDS - SAIDA' '19 - FDS - D1' '11 - FDS - 06 HS' '16 - FDS - D3 (R)'
 '09 - NOTURNA - PLANTAO' '20 - FDS - PLANTAO'
 '02 - SEMANA - AV.PRE ANESTESICA' '04.3 - SEMANA - COBERTURA EXTRA'
 '04.4 - SEMANA - COBERTURA EXTRA' '18 - FDS - D2 (R)'
 '02.1 - SEMANA - AV.PRE ANESTESICA' '15 - FDS - D3 (B)'
 '06 - SEMANA - ENDOSCOPIA' 'EINSTEIN' '12 - FDS - AV.PRE ANESTESICA'
 '03 - SEMANA - AMBULATORIO' '17 - FDS - D2 (B)' 'BLANC']
['01' '04' '04.1' '07' '08' '13' '14' '19' '11' '16' '09' '20' '02' '04.3'
 '04.4' '18' '02.1' '15' '06' 'EINSTEI' '12' '03' '17' 'BLAN']


In [33]:
tabelona['Nome'] = tabelona['Nome'].apply(lambda nome: unidecode.unidecode(nome).upper().strip().replace("  ", " "))
tabelona_backup = tabelona.copy

print(tabelona.shape)
tabelona.sample(5)

(7355, 13)


,Nome,Dia,Unidade,Periodo,Semana,Dia Semana,Entrada,Saida,ordem,data,PERIODO,periodo,setor_inicio
3980,MARCELA APARECIDA CORREA MARTINS,22,ANALIA,04.1 - SEMANA - CENTRO CIRURGICO,4,1,13:00,19:00,2,22/07/2024,tarde,3.0,04.1
135,ALEXANDRE BERNARDINO,24,VILLA,04 - SEMANA - CENTRO CIRURGICO,4,3,11:00,13:00,1,24/07/2024,manha,2.0,04
4653,MILTON OLAIO NETO,19,ITAIM,01 - NOTURNA - ENTRADA ANTES DAS 07HS,3,5,05:00,07:00,1,19/07/2024,6h,1.0,01
1800,FELIPE DE OLIVEIRA NUNES,30,SANTA ISABEL,04 - SEMANA - CENTRO CIRURGICO,5,2,07:00,19:00,1,30/07/2024,manha,2.0,04
2949,JULIANA SANTOS SABOIA,25,JABAQUARA,08 - NOTURNA - DISTANCIA,4,4,19:00,07:00,3,25/07/2024,noturno saida,4.0,08


# =======================================================================

# Diminuindo Tempos & Deslocado

In [34]:
from selenium.webdriver.common.action_chains import ActionChains
action_chains = ActionChains(browser)

## Pareamento

In [35]:
df['oque_fazer_com_o_registro'] = np.nan

for i in range(len(df)):

    nome = df['nome'][i]
    data = df['data'][i]
    periodo = df['PERIODO'][i]

    tabela = tabelona[(tabelona['Nome'] == nome) & 
                    (tabelona['data'] == data) &
                    (tabelona['PERIODO'] == periodo)]
    if len(tabela) == 0:
        df['oque_fazer_com_o_registro'][i] = 'criar plantao'

    elif len(tabela) == 1: # Verificando se os horarios batem
        if df['ENTRADA'][i] != tabela['Entrada'][tabela.index[0]] or df['SAIDA'][i] != tabela['Saida'][tabela.index[0]]:
            df['oque_fazer_com_o_registro'][i] = 'alterar horario'
        else:
            df['oque_fazer_com_o_registro'][i] = 'nada' # OK

    elif len(tabela) >= 2:
        df['oque_fazer_com_o_registro'][i] = 'apagar duplicata'
    else:

        print("WTF ?")
        break

count_freq(df, 'oque_fazer_com_o_registro')

,Freq,Perc
oque_fazer_com_o_registro,,
criar plantao,210,42.596349
nada,167,33.874239
alterar horario,114,23.123732
apagar duplicata,2,0.405680


In [36]:
df_filter = df[df['oque_fazer_com_o_registro'] == 'nada']
salvando_nome_arquivo('Feitos.xlsx', df_filter)

df_filter = df[df['oque_fazer_com_o_registro'] == 'alterar horario']
os.remove("df_gabs.xlsx")
salvando_nome_arquivo('df_gabs.xlsx', df_filter)

df_filter = df[df['oque_fazer_com_o_registro'] == 'criar plantao']
salvando_nome_arquivo('df_gabs2.xlsx', df_filter)

df_filter = df[df['oque_fazer_com_o_registro'] == 'apagar duplicata']
salvando_nome_arquivo('df_giov.xlsx', df_filter)

df = df[df['oque_fazer_com_o_registro'] == 'alterar horario'].reset_index(drop=True)
df_backup = df.copy()

4096  +=  167 => 4263
114
210
41  +=  2 => 43


In [37]:
count = 0
print(df_backup.shape)
df_backup.head()

(114, 20)


,chat,dia,nome,mensagem,controle,data,unidade,extra,tipo,ENTRADA,SAIDA,DURAÇÃO,PERIODO,periodo,setor,sem_ou_fds,unidade2,setor_antigo,setor_inicio,oque_fazer_com_o_registro
0,[24/07/2024 13:14:16] Alexandre Amaral Lucas: ...,24/07/2024,ALEXANDRE AMARAL LUCAS,24/07 OSASCO - ALEXANDRE AMARAL SAÍDA 13:11,NaN,24/07/2024,OSASCO,NaN,SAIDA,13:00,13:30,00:30,tarde,3,04.1 - SEMANA - CENTRO CIRURGICO,semana,OSASCO,SEMANA - TARDE - C.CIRURGICO,04.1,alterar horario
1,[24/07/2024 18:35:55] Alexandre Bernardino: 24...,24/07/2024,ALEXANDRE BERNARDINO,24/07 VILLA - ALEXANDRE - SAÍDA 18:30,NaN,24/07/2024,VILLA,NaN,SAIDA,13:00,18:30,05:30,tarde,3,04.1 - SEMANA - CENTRO CIRURGICO,semana,VILLA,SEMANA - TARDE - C.CIRURGICO,04.1,alterar horario
2,[23/07/2024 14:22:45] Aline Franchini Goulart ...,23/07/2024,ALINE FRANCHINI GOULART LIMA,23/07 VILLA ALINE SAÍDA 14:20,NaN,23/07/2024,VILLA,NaN,SAIDA,13:00,14:30,01:30,tarde,3,04.1 - SEMANA - CENTRO CIRURGICO,semana,VILLA,SEMANA - TARDE - C.CIRURGICO,04.1,alterar horario
3,[23/07/2024 14:48:47] Ana Clara Mourão Barreto...,23/07/2024,ANA CLARA MOURAO BARRETO,23/07 JABAQUARA CLARA - SAIDA 14:40,NaN,23/07/2024,JABAQUARA,NaN,SAIDA,13:00,15:00,02:00,tarde,3,04.1 - SEMANA - CENTRO CIRURGICO,semana,JABAQUARA,SEMANA - TARDE - C.CIRURGICO,04.1,alterar horario
4,[24/07/2024 14:02:43] Ana Jung Docko: 24/07 An...,24/07/2024,ANA JUNG DOCKO,24/07 ANALIA ANA SAÍDA 14:00,NaN,24/07/2024,ANALIA,NaN,SAIDA,13:00,14:00,01:00,tarde,3,04.1 - SEMANA - CENTRO CIRURGICO,semana,ANALIA,SEMANA - TARDE - C.CIRURGICO,04.1,alterar horario


# Execução

In [41]:
############################-CODIGO-############################
#Para cada profissional na lista
df = df_backup[count:]

for i in df['nome'].unique():
    print('='*20, i, '='*20)
    df_nome = df[df['nome'] == i]
    df_nome = df_nome.reset_index(drop=True)

    ###########################################
    # MUDANDO O PROFISSIONAL ESCOLHIDO
    subir_pg(); sleep(0.5)
    xpath_click('//*[@id="s2id_ddlProfessionals"]/a/span[1]'); sleep(1)
    xpath_estranho(i)
    xpath_click('//*[@id="select2-drop"]/ul/li/div')
    ###########################################
    
    # esperando carregamento do site
    WebDriverWait(browser, 30).until_not(visibility_of(xpath_element('//*[@id="loader"]')))
    sleep(2)
    # Capturando calendario de registros do individuo
    tabela = funcao(); sleep(0.3)
    
    # Para cada registro
    for t in range(len(df_nome)):
        ##########################################################################################################
        print(df_nome['mensagem'][t])

        '''
        # Analisandos os registros no **dia**
        dia_sel = tabela[tabela['data'] == df_nome['data'][t]] # & (tabela['Periodo'] == df_nome["setor"][t])]
        # Encurtando entrada - quando entrou atrasado
        if df_nome['PERIODO'][t] == 'manha' and df_nome['sem_ou_fds'][t] == 'fds':
            dia_sel = dia_sel[dia_sel['Entrada'] == '07:00']
        else:
            dia_sel = dia_sel[dia_sel['PERIODO'] == df_nome["PERIODO"][t]]
        '''

        ####################################################
        dia_sel = tabela[(tabela['data'] == df_nome['data'][t]) & (tabela['PERIODO'] == df_nome['PERIODO'][t])]
        ####################################################

        dia_sel.reset_index(drop=True, inplace=True)        
        ##########################################################################################################

        if len(dia_sel) == 0:
            df_backup['controle'][count] = "PROB"; print(count, "-> PROB")
        elif len(dia_sel) == 2:
            # abrindo/selecionando periodo conforme localização
            try:
                xpath_localizacao = '//*[@id="shift-monthly"]/div[2]/table/tbody/tr['+str(dia_sel['Semana'][0])+']/td['+str(dia_sel['Dia Semana'][0])+']/ul['+str(dia_sel['ordem'][0])+']/li'
                esperar_elemento_aparecer(xpath_localizacao, 5)
                xpath_click(xpath_localizacao)
            except:
                xpath_localizacao = '//*[@id="shift-monthly"]/div[2]/table/tbody/tr['+str(dia_sel['Semana'][0])+']/td['+str(dia_sel['Dia Semana'][0])+']/ul/li['+str(dia_sel['ordem'][0])+']'
                esperar_elemento_aparecer(xpath_localizacao, 5)
                xpath_click(xpath_localizacao)
            # Esperar componentes carregarem
            WebDriverWait(browser, 10).until(visibility_of(xpath_element('//*[@id="service-ckb-list-all-owner"]')))
            sleep(0.3)
            
            #apagando_registro(); sleep(0.3)
            tipo_registro_func('DISPENSADO'); sleep(0.5); atualizando_plantao()
            
            df_backup['controle'][count] = "OK"; print(count, "-> OK")
            WebDriverWait(browser, 10).until(visibility_of(xpath_element('//*[@id="s2id_ddlProfessionals"]/a/span[1]')))
            sleep(1)
        else:
            #abrindo/selecionando periodo conforme localização
            try:
                xpath_localizacao = '//*[@id="shift-monthly"]/div[2]/table/tbody/tr['+str(dia_sel['Semana'][0])+']/td['+str(dia_sel['Dia Semana'][0])+']/ul['+str(dia_sel['ordem'][0])+']/li'
                esperar_elemento_aparecer(xpath_localizacao, 5)
                xpath_click(xpath_localizacao)
            except:
                xpath_localizacao = '//*[@id="shift-monthly"]/div[2]/table/tbody/tr['+str(dia_sel['Semana'][0])+']/td['+str(dia_sel['Dia Semana'][0])+']/ul/li['+str(dia_sel['ordem'][0])+']'
                esperar_elemento_aparecer(xpath_localizacao, 5)
                xpath_click(xpath_localizacao)
            #Esperar componentes carregarem
            WebDriverWait(browser, 10).until(visibility_of(xpath_element('//*[@id="service-ckb-list-all-owner"]')))
            sleep(1)
            hora(df_nome['ENTRADA'][t], df_nome['DURAÇÃO'][t])
            if dia_sel['Unidade'][0] != df_nome['unidade'][t]:
                setor(df_nome['unidade'][t], df_nome['setor'][t])
            else:
                pass
            
            tipo_do_registro(df_nome['DURAÇÃO'][t], df_nome['sem_ou_fds'][t], df_nome['extra'][t])
            sleep(0.3)
            reset_selecao2()
            #salvar plantão
            sleep(0.3)
            atualizando_plantao(); atualizando_plantao(); sleep(2)
            df_backup['controle'][count] = "OK"; print(count, "-> OK")

            xpath_atualizar = '/html/body/div[5]/div/div[1]/div[2]/button[5]'
            try:
                if WebDriverWait(browser, 10).until_not(EC.visibility_of_element_located((By.XPATH, xpath_atualizar))) == False:
                    pass
            except:
                atualizando_plantao()
                try: 
                    if WebDriverWait(browser, 10).until_not(EC.visibility_of_element_located((By.XPATH, xpath_atualizar))) == False:
                        pass
                except:
                    break

            sleep(1)
            
        count += 1 
        
        if count % 25 == 0:
            print('='*80, '\n', df_backup['controle'].value_counts(), '\n'); df_backup.to_excel("df_gabs.xlsx", index=False)
        else:
            if t != len(df_nome)-1:
                print('='*20)
            
print("="*50, 'TERMINOU', "="*50)
print('Total  ', len(df_backup))
print('='*80, df_backup['controle'].value_counts(), '='*80); df_backup.to_excel("df_gabs.xlsx", index=False)

==================== ALEXANDRE AMARAL LUCAS ====================
24/07 OSASCO  -  ALEXANDRE AMARAL SAÍDA 13:11
0 -> OK
==================== ALEXANDRE BERNARDINO ====================
24/07 VILLA  -  ALEXANDRE  -  SAÍDA 18:30
duração >5 -> 100 pontos
1 -> OK
==================== ALINE FRANCHINI GOULART LIMA ====================
23/07 VILLA ALINE SAÍDA 14:20
2 -> OK
==================== ANA CLARA MOURAO BARRETO ====================
23/07 JABAQUARA CLARA  -  SAIDA 14:40
3 -> OK
==================== ANA JUNG DOCKO ====================
24/07 ANALIA ANA SAÍDA 14:00
4 -> OK
==================== ANDRE DA COSTA AGUIAR VENTURA ====================
24/07 MORUMBI VENTURA SAIDA 17:10
5 -> OK
==================== ANDREIA LOUREIRO MORI ====================
24/07 JABAQUARA ANDRÉIA SAÍDA 17:36
duração >5 -> 100 pontos
6 -> OK
==================== ANTONIO ALVARO DIAS LIMA DA CUNHA NETO ====================
23/07 CENTRAL  -  ÁLVARO SAÍDA 18:00
duração >5 -> 100 pontos
7 -> OK
==================== ARTHUR R

In [ ]:
WebDriverWait(browser, 10).until(visibility_of(xpath_element('//*[@id="service-ckb-list-all-owner"]')))

In [ ]:
df_backup.loc[int(count), 'controle'] = "PROB"
count += 1
print(df_backup['nome'][count])
print(count)

In [ ]:
count += 1

In [ ]:
count += 1

In [ ]:
df['nome'][df['nome'] == 'ICARO MOURA DE SOUSA'] = 'ICARO DE MOURA SOUSA'

## Salvando

In [42]:
df_corrigidos = df_backup[df_backup['controle'] == "OK"]
print("probs ok:", len(df_corrigidos))

df_prob = df_backup[df_backup['controle'] != "OK"]
print("probs:", len(df_prob))

###############################################################

os.remove("df_gabs.xlsx")
salvando_nome_arquivo('Feitos.xlsx', df_corrigidos)
print('.')
salvando_nome_arquivo('df_gabs2.xlsx', df_prob)

probs ok: 114
probs: 0
4263  +=  114 => 4377
.
210  +=  0 => 210


# ============================

# Plantões Normais

In [43]:
df = pd.read_excel('df_gabs2.xlsx')
df = df.sort_values(['nome', 'data']).reset_index(drop=True)
print(df.shape)
display(df.head())

(210, 20)


,chat,dia,nome,mensagem,controle,data,unidade,extra,tipo,ENTRADA,SAIDA,DURAÇÃO,PERIODO,periodo,setor,sem_ou_fds,unidade2,setor_antigo,setor_inicio,oque_fazer_com_o_registro
0,[24/07/2024 20:19:56] Alessandre Luiz Braga Ju...,24/07/2024,ALESSANDRE LUIZ BRAGA JUNIOR,24/07 ANALIA - ALESSANDRE SAIDA 20:18,NaN,24/07/2024,ANALIA,NaN,SAIDA,19:00,20:30,01:30,noturno saida,4,07 - NOTURNA - SAIDA APOS 19HS,semana,ANALIA,SEMANA - NOTURNO - SAIDA,7.0,criar plantao
1,[24/07/2024 13:14:03] Alexandre Amaral Lucas: ...,24/07/2024,ALEXANDRE AMARAL LUCAS,24/07 OSASCO - ALEXANDRE AMARAL ENTRADA 05:40,NaN,24/07/2024,OSASCO,NaN,ENTRADA,05:30,07:00,01:30,6h,1,01 - NOTURNA - ENTRADA ANTES DAS 07HS,semana,OSASCO,SEMANA - 6HS,1.0,criar plantao
2,[24/07/2024 06:15:37] Amanda Kiyoko Souza Koss...,24/07/2024,AMANDA KIYOKO SOUZA KOSSAKA,24/07 MORUMBI - AMANDA ENTRADA 06:00,NaN,24/07/2024,MORUMBI,NaN,ENTRADA,06:00,07:00,01:00,6h,1,01 - NOTURNA - ENTRADA ANTES DAS 07HS,semana,MORUMBI,SEMANA - 6HS,1.0,criar plantao
3,[24/07/2024 20:17:41] Amanda Kiyoko Souza Koss...,24/07/2024,AMANDA KIYOKO SOUZA KOSSAKA,24/07 MORUMBI - AMANDA SAÍDA 20:15,NaN,24/07/2024,MORUMBI,NaN,SAIDA,19:00,20:30,01:30,noturno saida,4,07 - NOTURNA - SAIDA APOS 19HS,semana,MORUMBI,SEMANA - NOTURNO - SAIDA,7.0,criar plantao
4,[25/07/2024 05:23:05] André da Costa Aguiar Ve...,25/07/2024,ANDRE DA COSTA AGUIAR VENTURA,25/07 MORUMBI VENTURA ENTRADA 5:20,NaN,25/07/2024,MORUMBI,NaN,ENTRADA,05:00,07:00,02:00,6h,1,01 - NOTURNA - ENTRADA ANTES DAS 07HS,semana,MORUMBI,SEMANA - 6HS,1.0,criar plantao


In [44]:
len(df)

210

In [45]:
H=0

# Execução

In [47]:
count = 0
for h in range(H, len(df)):
    #####################################################################################################################
    if h > 0:
        if df['nome'][h] != df['nome'][h-1] or count == 0:
            print('='*20, df['nome'][h], '='*20)
            count += 1
    #####################################################################################################################
    #try:
    print('[',h,'] |','dia:', df['data'][h],'| mensagem:', df['mensagem'][h])
    
    #click criar novo plantão
    criar_plantao()
    
    #SETOR - UNIDADE
    #esta disponivel o botão de setor ?
    esperar_elemento_aparecer('//*[@id="s2id_ddlServiceSector"]/a/span[1]', 10)
    #Selecionando Setor
    setor(df['unidade'][h], df['setor'][h])
    
    #PROFISSIONAL
    #esta disponivel o botão de profissional
    esperar_elemento_aparecer('//*[@id="service-ckb-list-all-owner"]', 10)
    nome_func(df['nome'][h])
    sleep(1)

    #DATA DO PLANTÃO
    data_func(df['data'][h])
    
    #INICIO DO PLANTÃO -- DURAÇÃO DO PLANTÃO
    hora(df['ENTRADA'][h], df['DURAÇÃO'][h])
    sleep(1)
        
    #TIPO
    tipo_do_registro(df['DURAÇÃO'][h], df['sem_ou_fds'][h], df['extra'][h])
    
    #SALVAR PLANTÃO
    sleep(1)
    salvar_plantao()
    salvar_plantao()
    salvar_plantao()
    sleep(4)

    #VERIFICANDO SE HOUVE ERRO - HÁ PLANTÃO NO MESMO HORARIO
    chatao = 5 #6 5 4
    if xpath_element('/html/body/div['+str(chatao)+']/div/div[2]/div[1]').text != "":
        fechar()
        df.loc[int(h), 'controle'] = "PROBS"; print("PROB")
    else:
        df.loc[int(h), 'controle'] = "NORM"; print("NORM")
    #####################################################################################################################
    '''except Exception as e:
        print(f"Erro ao processar o arquivo: {e}")
        if existencia('//*[@id="modal-add-service"]/div/div[1]/button'):
            fechar()
            print('*'*50)
            df.loc[int(h), 'controle'] = "ERRO_REPETIR"; print("ERRO_REPETIR")
            print('*'*50)
        else:
            print('-VERIFICAR-'*10)
            break'''
            
    #####################################################################################################################
    # praticamente um # finally
    WebDriverWait(browser, 10).until_not(visibility_of(xpath_element('//*[@id="modal-add-service"]/div/div[1]/button')))
    H += 1
    #####################################################################################################################
    if H % 25 == 0:
        print('='*80, '\n', df['controle'].value_counts(), '\n', '='*80); df.to_excel("df_gabs2.xlsx", index=False)
    if df['nome'][h] == df['nome'][h+1]:
        print('='*10)

print("="*20, 'TERMINOU', "="*20)
print('Total  ', len(df))
print('='*80, '\n', df['controle'].value_counts(), '\n', '='*80); df.to_excel("df_gabs2.xlsx", index=False)

==================== TERMINOU ====================
Total   210
 controle
NORM     198
PROBS     12
Name: count, dtype: int64 


In [ ]:
criar_plantao()

In [ ]:
#df.loc[int(h), 'controle'] = "NORM"; print("NORM")
df.loc[int(h), 'controle'] = "PROBS"; print("PROB")
H += 1

In [ ]:
df['nome'][df['nome'] == '~ ICARO MOURA'] = 'icaro de MOURA SOUSA'.upper()

## Salvando

In [48]:
print(df['controle'].value_counts())
print(df['controle'].value_counts(normalize=True)*100)

count_freq(df, 'controle')

controle
NORM     198
PROBS     12
Name: count, dtype: int64
controle
NORM     94.285714
PROBS     5.714286
Name: proportion, dtype: float64


,Freq,Perc
controle,,
NORM,198,94.285714
PROBS,12,5.714286


In [49]:
df_corrigidos = df[df['controle'] == "NORM"]
print("probs ok:", len(df_corrigidos))

df_prob = df[df['controle'] != "NORM"]
print("probs:", len(df_prob))

os.remove("df_gabs2.xlsx")
###############################################################

salvando_nome_arquivo('Feitos.xlsx', df_corrigidos)
print('.')
salvando_nome_arquivo('df_giov2.xlsx', df_prob)

probs ok: 198
probs: 12
4377  +=  198 => 4575
.
12


# =======================

# Dispensados

In [50]:
df_backup = pd.read_excel('df_dispensados.xlsx')
df_backup = df_backup.drop_duplicates(subset=['chat'])
df_backup = df_backup.sort_values(['nome','data']).reset_index(drop=True)

print(df_backup.shape)
df_backup.head()

(7, 7)


,chat,dia,nome,mensagem,controle,data,hora
0,[24/07/2024 15:07:24] Antonio Alvaro Dias Lima...,24/07/2024,ANTONIO ALVARO DIAS LIMA DA CUNHA NETO,24/07 CENTRAL - ÁLVARO DISPENSADO,OK,24/07/2024,NaN
1,[23/07/2024 18:46:21] Diego da Cunha Tomaz: 23...,23/07/2024,DIEGO DA CUNHA TOMAZ,23/07 SANTA - DIEGO DISPENSADO,OK,23/07/2024,NaN
2,[23/07/2024 20:57:07] Gabriela Meireles Linhar...,23/07/2024,GABRIELA MEIRELES LINHARES DE ARAUJO,22/07 ANÁLIA G.LINHARES DISPENSADO,OK,22/07/2024,NaN
3,[23/07/2024 18:49:54] Juliana Akemi Funabashi:...,23/07/2024,JULIANA AKEMI FUNABASHI,24/07 VILLA - AKEMI DISPENSADO,OK,24/07/2024,NaN
4,[24/07/2024 08:40:43] Matheus de Alencar Arrae...,24/07/2024,MATHEUS DE ALENCAR ARRAES,24/07 ITAIM ARRAES DISPENSADO,OK,24/07/2024,NaN


In [51]:
count = 0
df_backup['controle'] = np.nan

# Execução

def apagando_registro():
    chatao = str(5) #4 ou 5
    #Apagando
    browser.find_element("xpath", '/html/body/div['+chatao+']/div/div[1]/div[2]/button[1]').click()
    #Clicando em continuar
    sleep(0.5)
    browser.find_element("xpath", '/html/body/div['+chatao+']/div/div[1]/div[3]/button[2]').click()
apagando_registro()

In [52]:
len(df_backup)

7

In [57]:
#Para cada profissional na lista
df = df_backup[count:]

for i in df['nome'].unique():
    print('='*20, i, '='*20)
    df_nome = df[df['nome'] == i]
    df_nome.reset_index(drop=True, inplace=True)
    
    xpath_click('//*[@id="s2id_ddlProfessionals"]/a/span[1]'); sleep(1)
    xpath_estranho(i)
    xpath_click('//*[@id="select2-drop"]/ul/li/div')

    #esperando carregamento do site
    WebDriverWait(browser, 30).until_not(visibility_of(xpath_element('//*[@id="loader"]')))

    tabela = funcao()    
    sleep(2)
    
    #Para cada registro na lista_nome
    for t in range(len(df_nome)):
        print(df_nome['mensagem'][t])
        dia_sel = tabela[tabela['data'] == df_nome['data'][t]]
        dia_sel.reset_index(drop=True, inplace=True)
        
        if len(dia_sel) == 0:
            print(count, "-> Ja apagado")
            print(df_nome['chat'][t] + " - " + df_nome['data'][t])
            df_backup['controle'][count] = "OK"
        else:
            for r in range(len(dia_sel)-1, -1, -1):
                print('PRINCIPAL')
                #abrindo/selecionando periodo conforme localização
                dia_ordem = r
                xpath_textao = '//*[@id="shift-monthly"]/div[2]/table/tbody/tr['+str(dia_sel['Semana'][0])+']/td['+str(dia_sel['Dia Semana'][0])+']/ul['+str(dia_sel['ordem'][dia_ordem])+']/li'
                xpath_click(xpath_textao)
                #Esperar componentes carregarem
                WebDriverWait(browser, 10).until(visibility_of(xpath_element('//*[@id="service-ckb-list-all-owner"]')))
                sleep(2)
                reset_selecao2()
                tipo_registro_func('DISPENSA')
                sleep(0.3)
                reset_selecao2()
                #salvar plantão
                sleep(0.3)
                atualizando_plantao(); atualizando_plantao(); sleep(2)
                try:
                    if WebDriverWait(browser, 10).until_not(EC.visibility_of_element_located((By.XPATH, xpath_atualizar))) == False:
                        pass
                except:
                    atualizando_plantao()
                    try: 
                        if WebDriverWait(browser, 10).until_not(EC.visibility_of_element_located((By.XPATH, xpath_atualizar))) == False:
                            pass
                    except:
                        break
            
            tabela = funcao() #atualizando tabela de registros
            sleep(2)
            print(count, '-> OK')
            print(df_nome['chat'][t] + " - " + df_nome['data'][t])
            df_backup['controle'][count] = "OK"  
            
        count += 1 
        if count % 25 == 0:
            print('='*80, '\n', df_backup['controle'].value_counts(), '\n', '='*80); df_backup.to_excel("df_dispensados.xlsx", index=False)
        else:
            if t != len(df_nome)-1:
                print('='*20)
    
print('='*25, 'TERMINOU', '='*25)
print('Total  ', len(df_backup))
print('='*80, df_backup['controle'].value_counts(), '='*80); df_backup.to_excel("df_dispensados.xlsx", index=False)

========================= TERMINOU =========================
Total   7
================================================================================ controle
OK    7
Name: count, dtype: int64 ================================================================================


In [ ]:
count += 1 

In [ ]:
/html/body/form/main/div[2]/div[2]/div/div[1]/div[1]/div/a

In [56]:
df_backup['controle'][count] = "OK"
count += 1

In [ ]:
df_backup['nome'][df_backup['nome'] == "ICARO MOURA DE SOUSA"] = "ICARO DE MOURA SOUSA"

## Salvando

In [58]:
df_corrigidos = df_backup[df_backup['controle'] == "OK"]
print("probs ok:", len(df_corrigidos))

df_prob = df_backup[df_backup['controle'] != "OK"]
print("probs:", len(df_prob))

################################################################

os.remove("df_dispensados.xlsx")
salvando_nome_arquivo('Feitos.xlsx', df_corrigidos)
print('.')
salvando_nome_arquivo('df_giov.xlsx', df_prob)

probs ok: 7
probs: 0
4575  +=  7 => 4582
.
43  +=  0 => 43


# =============================

# Encerrando Automação

In [59]:
browser.quit()

# ==============================

# Residuos Finais

In [60]:
df_feito = pd.read_excel("Feitos.xlsx")
print(len(df_feito))

df_prob0 = pd.read_excel("df_giov.xlsx")
print(len(df_prob0))
os.remove("df_giov.xlsx")

df_prob01 = pd.read_excel("df_giov2.xlsx")
print(len(df_prob01))
os.remove("df_giov2.xlsx")

df_prob = pd.concat([df_prob0, df_prob01])
df_prob = df_prob.drop_duplicates(subset=['chat'])
salvando_nome_arquivo('df_prob.xlsx', df_prob)

print(len(df_prob))

4582
43
12
55
55


In [61]:
total = len(df_feito) + len(df_prob)

percent = len(df_feito) / total
percent = round(percent*100, 2)
percent = str(percent) + "%"
percent = "("+percent+")"
print("Registros App:", len(df_feito), percent)

percent = len(df_prob) / total
percent = round(percent*100, 2)
percent = str(percent) + "%"
percent = "("+percent+")"
print("Registros Giovanelli", len(df_prob), percent)
print("#"*30, "\n")

for nome in sorted(list(df_prob['nome'].unique())):
    print(nome, "="*10)
    selecao = df_prob.loc[df_prob['nome']==nome]
    selecao.reset_index(drop=True, inplace=True)
    
    for i in range(len(selecao)):
        print(selecao["mensagem"][i])
    
    print("\n")

Registros App: 4582 (98.81%)
Registros Giovanelli 55 (1.19%)
############################## 

ALEXANDRE BERNARDINO ==========
23/07 ANÁLIA  -  ALEXANDRE  -  SAÍDA 19:00


AMANDA KIYOKO SOUZA KOSSAKA ==========
23/07 MORUMBI  -  AMANDA SAÍDA 23:40


ANA CLARA MOURAO BARRETO ==========
24/07 OSASCO CLARA 19:30


ANNA BEATRIZ ARANHA TREVISAN ==========
23/07 VILLA  -  ANNA BEATRIZ SAÍDA 18:00
24/07 VILLA  -  ANNA BEATRIZ SAÍDA 17:40


BEATRIZ ANTIPOU DOS SANTOS ==========
24/07 -  MATERNIDADE -  SAÍDA 07:00  ( PLANTÃO ) 


BIANCA PRISCILA BUNIOTTI ==========
24/07 JABAQUARA  -  BIANCA ENTRADA 07:00  ( DISTÂNCIA ) 


BRIAN SOSSAI PACHECO ==========
24/07 ITAIM  -  BRIAN SAÍDA 07:00  ( PLANTÃO ) 


BRUNO FERRAZ DE CAMPOS BARONE ==========
24/07 VNS BARONE ENTRADA 07:00  ( DISTÂNCIA ) 


BRUNO STORNIOLI TAGUCHI LEMOS ==========
24/07 HIAE BRUNO LEMOS ENTRADA 12:00  ( EXTERNAS )  ‎<MENSAGEM EDITADA>
24/07 HIAE BRUNO LEMOS SAÍDA 17:45


CAMILA CHIARANTANO HARRISON ==========
24/07 VILLA CAMILA

In [ ]:
os.remove("df_prob.xlsx")
df_prob.to_excel("df_prob_enviado_2407.xlsx", index=False)

# =================================

# Teste